In [ ]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

In [ ]:
scope.gain.gain = 55
scope.gain.mode = 'high'
scope.adc.samples = 5000
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7372800
scope.clock.adc_src = "clkgen_x4"
scope.clock.adc_phase = 128
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.tio3 = True #Run condition
scope.io.pdic = False #SPI forwarding off
scope.io.hs2 = "clkgen"

target.baud = 38400

print("\n***ESP 32 Target Usage Notes***")
print("   Call set_bootloader to enter BOOTLOADER so external USB-Serial cable will work.")
print("   Call set_run to enter RUN mode.")
print("   Jumper TDI to GND to disable boot-up messages (reduce noise if looking at boot decryption).")


def set_bootloader():
    scope.io.tio3 = False
    scope.io.tio1 = None
    scope.io.tio2 = None
 
    #Toggle reset
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = None #Pulls high by default

def set_run():
    scope.io.tio3 = True
    scope.io.tio1 = "serial_rx"
    scope.io.tio2 = "serial_tx"
    
    #Toggle reset
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = None #Pulls high by default